In [9]:
import sqlite3 as lite
import os
import sys
import numpy as np
import collections
sys.path.insert(0, '../../scripts/')
jupyter_directory = os.getcwd()
from IPython.display import Image
import matplotlib.pyplot as plt
from collections import OrderedDict
import os

In [10]:
import analysis as an

In [11]:
# metrics to get
# does it transition?
# pu inventory
# raffinate
# lwr unf reprocessed
# reprocessing throughput
# unreprocessed lwr amount

In [12]:
# get isotopics for input and output recipe
def get_element_content(cur, commodity, z):
    
    query = cur.execute('SELECT * FROM transactions '
                        'INNER JOIN resources ON '
                        'resources.resourceid = transactions.resourceid '
                        'WHERE commodity = "%s"' %commodity).fetchall()
    for i in query:
        qualid = i['qualid']
        break
    query = cur.execute('SELECT NucId, MassFrac FROM compositions '
                        'WHERE qualid = %i' %qualid).fetchall()
    el_content = 0
    for row in query:
        if row['NucId'] // 1e7 == z:
            el_content += row['MassFrac']
    return el_content


In [15]:
def metric_solver(cur):
    output_dict = OrderedDict({})
    lwr_pu = get_element_content(cur, 'lwr_')
    sfr_pu = get_element_contenxt(cur, )
    sfr_used_pu = get_element_content(cur, )
    blanket_used_pu = get_element_content(cur, )
    try:
        legacy_pu = get_element_content(cur, )
    except:
        print('This one does not have legacy pu')
    
    # pu generated
    storage_id = an.get_agent_ids(cur, 'storage')
    rep_id = an.get_agent_ids(cur, 'separations')
    lwr_fuel_out = np.array(an.get_facility_commodity_flux(cur, storage_id, [''], False, True))
    sfr_fuel_out = np.array(an.get_facility_commodity_flux(cur, storage_id, [''], False, True))
    blanket = np.array(an.get_facility_commodity_flux(cur, storage_id, [''], False, True))
    try:
        legacy = np.array(an.get_facility_commodity_flux(cur, rep_id, [''], False, True))
    except:
        print('This one does not have legacy pu')
    pu_generated = (lwr_pu * lwr_fuel_out) + (sfr_fuel_out * sfr_used_pu) + (blanket * blanket_used_pu)
    try:
        pu_generate += legacy * legacy_pu
    except:
        print('I TOLD YOU NO LEGACY')

    # pu used
    mixer_id = an.get_agent_ids(cur, 'mixer')
    sfr_fuel_in = np.array(an.get_facility_commodity_flux(cur, mixer_id, [''], True, True))
    pu_used = (sfr_fuel_in * sfr_pu)
    
    # lwr reprocessed
    legacy_sfr = sfr_fuel_in['']
    legacy_pu = legacy_sfr * sfr_pu
    legacy_lwr_reprocessed = legacy_pu / lwr_pu
    output_dict['lwr UNF reprocessed'] = legacy_lwr_reprocessed[-1]
    
    # unreprocessed lwr amount
    lwr_amount = lwr_fuel_out[''][-1]
    try:
        lwr_amount += legacy[''][-1]
    except:
        print('NO LEGACY')
    unreprocessed_lwr = lwr_amount - legacy_lwr_reprocessed[-1]
    output_dict['unreprocessed lwr'] = unreprocessed_lwr
    
    # reprocessing throughput
    new_sfr = sfr_fuel_in['']
    new_sfr_reprocesed = new_sfr * sfr_pu / sfr_used_pu
    reprocessing_throughput = new_sfr_reprocessed + legacy_lwr_reprocesed
    output_dict['sfr UNF reprocessed'] = new_sfr_reprocessed[-1]

    # raffinate
    sink_id = an.get_agent_ids(cur, 'sink')
    raffinate = np.array(an.get_facility_commodity(cur, sink_id, ['reprocess_waste'], False, True))
    output_dict['raffinate'] = raffinate[-1]
    
    # get them metrics
    pu_inventory = pu_generated - pu_used
    output_dict['pu inventory'] = pu_inventory[-1]
    return output_dict

In [ ]:
# do work
f = open('./output/metrics', 'w')
for dire in directory:
    for k in ['.sqlite', '.sqlite', '.sqlite']:
        filename = './%s/%s' %(dire, k)
        outputname = '%s_%s' %(dire, k.replace('.sqlite', ''))
        print('DOING %s' %outputname)
        cur = an.get_cusor(filename)
        an.plot_power(cur)
        if k == '.sqlite':
            os.rename('power_plot.png', './output/%s.png' %(outputname))
        out_dict = metric_solver(cur)
        f.write('%s\n======================' %outputname)
        for key, val in out_dict.items():
            write_string = '%s:\t%s\ttons' %(key, str(val))
            f.write(write_string)
        f.write('\n')